![image](car.jpeg)

**Car-ing is sharing**, an auto dealership company for car sales and rental, is taking their services to the next level thanks to **Large Language Models (LLMs)**.

As their newly recruited AI and NLP developer, you've been asked to prototype a chatbot app with multiple functionalities that not only assist customers but also provide support to human agents in the company.

The solution should receive textual prompts and use a variety of pre-trained Hugging Face LLMs to respond to a series of tasks, e.g. classifying the sentiment in a car’s text review, answering a customer question, summarizing or translating text, etc.


In [105]:
# Import necessary packages
import os
import pandas as pd
import torch
import evaluate
from openai import OpenAI

from transformers import logging, pipeline
logging.set_verbosity(logging.WARNING)

from transformers import AutoModel, AutoTokenizer, Trainer, TrainingArguments
from transformers import AutoModelForSequenceClassification


# Sentiment Analysis of car reviews

In [104]:
reviews = pd.read_csv('car_reviews.csv', sep=';')
reviews.head()

sentiment_class = pipeline(task="sentiment-analysis", model="siebert/sentiment-roberta-large-english")

summary = []
for review in reviews['Review']:
    summary.append(sentiment_class(review))

pred_label = []
label_score = []
for i in range(len(summary)):
    pred_label.append(summary[i][0]['label'].upper())
    label_score.append(summary[i][0]['score'])

reviews['pred_class'] = pred_label
reviews['conf_pred_score'] = label_score

reviews


Device set to use cpu


,Review,Class,pred_class,conf_pred_score
0,I am very satisfied with my 2014 Nissan NV SL....,POSITIVE,POSITIVE,0.998915
1,The car is fine. It's a bit loud and not very ...,NEGATIVE,NEGATIVE,0.999505
2,"My first foreign car. Love it, I would buy ano...",POSITIVE,POSITIVE,0.998864
3,I've come across numerous reviews praising the...,NEGATIVE,NEGATIVE,0.999491
4,I've been dreaming of owning an SUV for quite ...,POSITIVE,POSITIVE,0.998922


## Model Evaluation Accuracy/F1

In [106]:
label_map = {'NEGATIVE':0, 'POSITIVE':1}

accuracy = evaluate.load('accuracy')
f1 = evaluate.load('f1')

accuracy_score = accuracy.compute(references=reviews['Class'].map(label_map), predictions=reviews['pred_class'].map(label_map))
f1_score = f1.compute(references=reviews['Class'].map(label_map), predictions=reviews['pred_class'].map(label_map))

print(f'{accuracy_score}\n{f1_score}')

{'accuracy': 1.0}
{'f1': 1.0}


# English to Spanish translaion LLM

In [125]:
first_review = reviews.loc[0,'Review']
first_two_sentences = first_review.split('.')[:2]
first_two_sentences

translation_llm = pipeline(
    "translation",
    model="facebook/nllb-200-distilled-600M",
    src_lang="eng_Latn",
    tgt_lang="spa_Latn"
)
translations = []
for sentence in first_two_sentences:
    translations.append(translation_llm(sentence))




C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\huggingface_hub\file_download.py:801: UserWarning: Not enough free disk space to download the file. The expected file size is: 2460.36 MB. The target location C:\Users\HP\.cache\huggingface\hub\models--facebook--nllb-200-distilled-600M\blobs only has 0.00 MB free disk space.
  warnings.warn(
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cpu


TypeError: can only concatenate list (not "str") to list

In [133]:
full_translation = translations[0][0]['translation_text']+'. '+translations[1][0]['translation_text']

print(full_translation )


Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta furgoneta para mis entregas de negocios y uso personal.
